In [1]:
#################################################################################
# Laden aller relevate Module
#################################################################################
import pyspark
from pyspark.sql.functions import *
import json
import csv
from datetime import datetime
from delta import *
import delta

# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

In [2]:
# first for local usage pip install delta-spark

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", "/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.driver.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.executor.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc=spark.sparkContext


In [3]:
spark

# Bespiel 1: Tabelle Drop, Filter, Sort

## Dataframe - Drop, Filter, Sort

In [5]:
df=(spark
    .read
    .format("csv")
    .option("header",True)
    .option("delimiter",";")
    .load("../2_lab/data/simple/")
   )

df.show(10)

+-------------+------+----------+
|         name|anzahl|      date|
+-------------+------+----------+
|      Bankura|    48|1544861923|
|    Beypazarı|    32|1576709032|
|      Otranto|    36|1573575340|
|    Balıkesir|    50|1567507084|
|     Dreieich|    28|1552948625|
|  Neunkirchen|    14|1546330649|
|Montes Claros|    44|1588255608|
|     Avellino|    33|1542196665|
|     Santiago|    40|1549252438|
|       Lavoir|    16|1596575768|
+-------------+------+----------+
only showing top 10 rows



In [6]:
df2=(df
     .drop("date")
     .where(col("anzahl")>40)
     .sort(col("name").desc())
    )

df2.show(5)

+--------------------+------+
|                name|anzahl|
+--------------------+------+
|           Warisoulx|    46|
|        Vizianagaram|    44|
|             Uberaba|    42|
|San Benedetto del...|    46|
|         Saint-Louis|    47|
+--------------------+------+
only showing top 5 rows



## RDD - Drop, Filter, Sort

In [7]:
rdd=sc.textFile("../2_lab/data/simple/")

# Zeilen von Strings
rdd.take(10)

['name;anzahl;date',
 'Bankura;48;1544861923',
 'Beypazarı;32;1576709032',
 'Otranto;36;1573575340',
 'Balıkesir;50;1567507084',
 'Dreieich;28;1552948625',
 'Neunkirchen;14;1546330649',
 'Montes Claros;44;1588255608',
 'Avellino;33;1542196665',
 'Santiago;40;1549252438']

In [12]:
header=rdd.first()
rdd2=(rdd
      # remove first line
      .filter(lambda line: line != header )
      # split string into array
      .map(lambda line: line.split(";"))
      # change second element in array to int, drittes Element entfernen
      .map(lambda x: [x[0],int(x[1])])
      # filter on second array element (anzahl) < 40
      .filter(lambda x: x[1]>40)
      # sortby first array element descending
      .sortBy(lambda x: x[0],False)
     )
rdd2.take(5)

[['Warisoulx', 46],
 ['Vizianagaram', 44],
 ['Uberaba', 42],
 ['San Benedetto del Tronto', 46],
 ['Saint-Louis', 47]]

In [13]:
rdd2.toDF(["name","anzahl"]).show(5)

+--------------------+------+
|                name|anzahl|
+--------------------+------+
|           Warisoulx|    46|
|        Vizianagaram|    44|
|             Uberaba|    42|
|San Benedetto del...|    46|
|         Saint-Louis|    47|
+--------------------+------+
only showing top 5 rows



# Beispiel 2: Word Count Fließtext

## RDD - reduce by key

In [14]:
rdd4=sc.textFile("../2_lab/data/word-count/es-ist-nacht.txt")

# Zeilen von Strings
rdd4.take(10)

['Es ist Nacht',
 'und mein Herz kommt zu dir',
 'hälts nicht aus',
 'hälts nicht aus mehr bei mir',
 'Legt sich dir auf die Brust',
 'wie ein Stein',
 'sinkt hinein',
 'zu dem deinen hinein',
 'Dort erst',
 'dort erst kommt es zur Ruh']

In [19]:
rdd5=(rdd4
      # splitte alles in Einzelworte und einen langen Datensatz
      .flatMap(lambda line: line.split(" "))
      # mape jedes Wort auf ein Tupel mit 1
      .map(lambda word: (word,1))
      # alles über Keys addieren/reduzieren
      .reduceByKey(lambda x,y: x+y)
      # über key sortieren
      .sortByKey(False)
      # über zweite Spalte sortieren
      .sortBy(lambda x: x[1],False)
)

rdd5.take(10)

[('zu', 2),
 ('nicht', 2),
 ('kommt', 2),
 ('hälts', 2),
 ('hinein', 2),
 ('erst', 2),
 ('dir', 2),
 ('aus', 2),
 ('zur', 1),
 ('wie', 1)]

### Dataframe explode und groupby

In [20]:
df4=(spark
    .read
    .format("text")
    .load("../2_lab/data/word-count/es-ist-nacht.txt")
   )

df4.show(10, truncate=False)

+----------------------------+
|value                       |
+----------------------------+
|Es ist Nacht                |
|und mein Herz kommt zu dir  |
|hälts nicht aus             |
|hälts nicht aus mehr bei mir|
|Legt sich dir auf die Brust |
|wie ein Stein               |
|sinkt hinein                |
|zu dem deinen hinein        |
|Dort erst                   |
|dort erst kommt es zur Ruh  |
+----------------------------+
only showing top 10 rows



In [23]:
df5=(df4
     # zeile in array splitten
     .withColumn("words",split(col("value")," "))
     # array in zeile exploden
     .withColumn("word",explode(col("words")))
     # nach gleichen Wörtern gruppieren und addieren
     .groupBy("word")
     .count()
     # sortieren nach count und word
     .sort(col("count").desc(),col("word").desc())
    )

df5.show(100,truncate=False)

+------+-----+
|word  |count|
+------+-----+
|zu    |2    |
|nicht |2    |
|kommt |2    |
|hälts |2    |
|hinein|2    |
|erst  |2    |
|dir   |2    |
|aus   |2    |
|zur   |1    |
|wie   |1    |
|und   |1    |
|sinkt |1    |
|sich  |1    |
|seines|1    |
|mir   |1    |
|mein  |1    |
|mehr  |1    |
|liegt |1    |
|ist   |1    |
|ewigen|1    |
|es    |1    |
|ein   |1    |
|dort  |1    |
|die   |1    |
|dem   |1    |
|deinen|1    |
|bei   |1    |
|auf   |1    |
|am    |1    |
|Stein |1    |
|Ruh   |1    |
|Nacht |1    |
|Legt  |1    |
|Herz  |1    |
|Grund |1    |
|Es    |1    |
|Du    |1    |
|Dort  |1    |
|Brust |1    |
+------+-----+

